In [15]:
import bisect

# open text file
def ReadFile(filename: str):
  with open(filename, 'r') as f:
    lines = f.readlines()
  return lines

class AlmanacMap(object):
  
  def __init__(self, sorted_from_values: list, map_from_value_to_to_value_and_count: dict):
    self._sorted_from_values = sorted_from_values
    self._map_from_value_to_to_value_and_count = map_from_value_to_to_value_and_count
    
  def Get(self, from_value: int):
    index = bisect.bisect(self._sorted_from_values, from_value)
    if index == 0:
      return from_value
    else:
      candidate_range_from_value = self._sorted_from_values[index - 1]
      candidate_range_end_value = candidate_range_from_value + self._map_from_value_to_to_value_and_count[candidate_range_from_value][1]
      if from_value >= candidate_range_from_value and from_value < candidate_range_end_value:
        return self._map_from_value_to_to_value_and_count[candidate_range_from_value][0] + (from_value - candidate_range_from_value)
      else:
        return from_value
  
  def ParseInput(lines: list):
    from_values = []
    map_from_value_to_to_value_and_count = {}
    for line in lines:
      to_str, from_str, count_str = line.split(' ')
      # convert to int
      from_value = int(from_str)
      to_value = int(to_str)
      count_value = int(count_str)
      # populate from_values
      from_values.append(from_value)
      # populate map
      if from_value not in map_from_value_to_to_value_and_count:
        map_from_value_to_to_value_and_count[from_value] = (to_value, count_value)
    from_values.sort()
    return __class__(from_values, map_from_value_to_to_value_and_count)


class Almanac(object):
  
  MAP_NAMES = [
    'seed-to-soil', 
    'soil-to-fertilizer',
    'fertilizer-to-water',
    'water-to-light',
    'light-to-temperature',
    'temperature-to-humidity',
    'humidity-to-location'
  ]
  
  def __init__(self, seed_values: list, map_of_maps: dict):
    self._seed_values = seed_values
    self._map_of_maps = map_of_maps
    
  def _CalculateSeedLocation(self, seed_value: int):
    value = seed_value
    for map_name in __class__.MAP_NAMES:
      value = self._map_of_maps[map_name].Get(value)
    return value
  
  def MinimumSeedLocation(self):
    return min([self._CalculateSeedLocation(seed_value) for seed_value in self._seed_values])

  def ParseInput(lines: list):
    index = 0
    
    # Parse the seeds.
    assert lines[index].startswith('seeds:')
    _, seeds = lines[index].split(':')
    seed_values = [int(x) for x in seeds.strip().split(' ')]
    
    index += 1
    assert lines[index].strip() == ''
    index += 1
    
    # Parse the maps.
    map_of_maps = {}
    for map_name in __class__.MAP_NAMES:
      assert lines[index].startswith(map_name + ' map:')
      index += 1
      map_lines = []
      while index < len(lines) and lines[index].strip() != '':
        map_lines.append(lines[index])
        index += 1
      else:
        index += 1
      map_of_maps[map_name] = AlmanacMap.ParseInput(map_lines)
      
    return __class__(seed_values, map_of_maps)

def SolvePartOne(input_file: str):
  lines = ReadFile(input_file)
  almanac = Almanac.ParseInput(lines)
  return almanac.MinimumSeedLocation()

assert SolvePartOne('sample.txt') == 35
part_one_solution = SolvePartOne('input.txt')
print('Part One Solution:', part_one_solution)
assert part_one_solution == 510109797

Part One Solution: 510109797
